In [10]:
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader

In [11]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

In [12]:
class Churnpredict:
    def __init__(self,X,y):
        self.X=X
        self.y=y
    
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [13]:
df=pd.read_csv('./final_dataset.csv')

X=df.drop(columns=['Churn']).values
y=df['Churn'].values

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

batch_size=50
learning_rate=0.01
dataset = Churnpredict(X, y)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size  # This ensures the split sums up to the total dataset length

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])


train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

examples = iter(train_loader)
sample, labels = next(examples)

In [14]:
class NeuralNet(nn.Module):
    def __init__(self,input_size, hidden_size, num_classes):
        super(NeuralNet,self).__init__()
        self.l1=nn.Linear(input_size,hidden_size)
        self.relu=nn.ReLU()
        self.l2=nn.Linear(hidden_size,hidden_size)
        self.relu2=nn.ReLU()
        self.l3=nn.Linear(hidden_size,hidden_size)
        self.relu3=nn.ReLU()
        self.l4=nn.Linear(hidden_size,num_classes)
        self.sigmoid=nn.Sigmoid()
        # self.dropout = nn.Dropout(p=0.5)

    def forward(self,x):
        out=self.l1(x)
        out=self.relu(out)
        out=self.l2(out)
        out=self.relu2(out)
        out=self.l3(out)
        out=self.relu3(out)
        out=self.l4(out)
        out=self.sigmoid(out)

        return out

In [15]:
input_size=5
hidden_size = 50
num_classes=1

model=NeuralNet(input_size=input_size,hidden_size=hidden_size,num_classes=num_classes)
model = model.to(device)

criterion=nn.BCELoss()
optimiser=torch.optim.SGD(model.parameters(),lr=learning_rate)

n_total_steps = len(train_loader)

num_epochs=100

for epoch in range(num_epochs):
    for i, (sample, labels) in enumerate(train_loader):
        sample=sample.to(device)
        labels = labels.to(device).view(-1, 1)

        # forward
        outputs = model(sample).to(device)
        loss = criterion(outputs, labels)

        # backward
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

        if (i + 1) % 100 == 0:
            print(
                f"epcoh{epoch + 1}/{num_epochs}, step: {i + 1}/{n_total_steps} loss: {loss.item():.4f}"
            )


epcoh1/100, step: 100/113 loss: 0.6641
epcoh2/100, step: 100/113 loss: 0.5930
epcoh3/100, step: 100/113 loss: 0.6246
epcoh4/100, step: 100/113 loss: 0.5671
epcoh5/100, step: 100/113 loss: 0.4971
epcoh6/100, step: 100/113 loss: 0.5877
epcoh7/100, step: 100/113 loss: 0.6288
epcoh8/100, step: 100/113 loss: 0.5935
epcoh9/100, step: 100/113 loss: 0.6951
epcoh10/100, step: 100/113 loss: 0.5732
epcoh11/100, step: 100/113 loss: 0.5731
epcoh12/100, step: 100/113 loss: 0.6136
epcoh13/100, step: 100/113 loss: 0.5934
epcoh14/100, step: 100/113 loss: 0.5935
epcoh15/100, step: 100/113 loss: 0.6552
epcoh16/100, step: 100/113 loss: 0.6748
epcoh17/100, step: 100/113 loss: 0.5528
epcoh18/100, step: 100/113 loss: 0.6549
epcoh19/100, step: 100/113 loss: 0.5935
epcoh20/100, step: 100/113 loss: 0.4916
epcoh21/100, step: 100/113 loss: 0.6138
epcoh22/100, step: 100/113 loss: 0.5732
epcoh23/100, step: 100/113 loss: 0.5935
epcoh24/100, step: 100/113 loss: 0.5935
epcoh25/100, step: 100/113 loss: 0.6752
epcoh26/1

In [16]:
with torch.no_grad():
    n_correct=0
    n_samples=0

    for sample,labels in test_loader:
        sample=sample.to(device)
        labels=labels.to(device)

        output=model(sample)
        #value,index
        _,predictions=torch.max(output,1)
        n_samples +=labels.shape[0]
        n_correct +=(predictions==labels).sum().item()
acc=100.0 * (n_correct/n_samples)
print(f'Accuracy: {acc}')

Accuracy: 73.52732434350602


In [ ]:
import torch
torch.save(model, "churn_model.pkl")
print("Model saved as churn_model.pkl")


Model saved as churn_model.pkl
